In [ ]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [3]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [4]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [5]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [6]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
tfidfvect.vocabulary_['cellphone']

26382

In [10]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [11]:
idx2word[26382]

'cellphone'

In [12]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [13]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [14]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [15]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [16]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [17]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [18]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [19]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [20]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [21]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [22]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [23]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

# **Consigna del desafío 1**

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

**3**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**4**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


# **Resolución del desafío**

In [24]:
# Cargamos train/test estandar del dataset 20 newsgroups
# cargamos los datos (ya separados de forma predeterminada en train y test)
train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

len(train.data), len(test.data), len(train.target_names)

(11314, 7532, 20)

## **1. Vectorización de documentos**

*Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.*

> Creamos un vectorizer de frecuencias y lo fiteamos en train, generando la "matriz de documento-termino"

In [73]:
import pandas as pd

class MyVectorizer:
  def __init__(self, X_train, y_train, target_map):
    self.fit(X_train, y_train, target_map)

  def fit(self, X_train, y_train, target_map):
    vectorizer = TfidfVectorizer()
    X_train = vectorizer.fit_transform(train.data)

    self.doc_term_matrix = X_train
    self.vectorizer = vectorizer
    self.idx2word = {v: k for k,v in vectorizer.vocabulary_.items()}
    self.labels = y_train
    self.target_names = target_map
    self.vocabulary_ = vectorizer.vocabulary_

    print(type(X_train))
    print(f'shape: {X_train.shape}')
    print(f'Cantidad de documentos: {X_train.shape[0]}')
    print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

    return vectorizer

  def transform(self, data):
    return self.vectorizer.transform(data)

  def get_index_from_vocab(self, word):
    return self.vectorizer.vocabulary_[word]

  def get_word_from_index(self, index):
    return self.idx2word[index]

  def get_doc_term_matrix(self):
    return self.doc_term_matrix

  def analyze_similarity(self, n_queries=5, topk=5, random_state=42):
    rng = np.random.default_rng(random_state)
    n_docs = self.doc_term_matrix.shape[0]
    query_indices = rng.choice(n_docs, size=n_queries, replace=False)

    sims = cosine_similarity(self.doc_term_matrix, self.doc_term_matrix)
    results = []

    for qidx in query_indices:
        sim_scores = sims[qidx].copy()
        sim_scores[qidx] = -np.inf  # excluir el propio documento
        top_idx = np.argpartition(-sim_scores, kth=topk)[:topk]
        top_idx = top_idx[np.argsort(-sim_scores[top_idx])]

        for rank, i in enumerate(top_idx, start=1):
            results.append({
                "query_idx": qidx,
                "query_label": self.target_names[self.labels[qidx]] if self.labels is not None else None,
                "cand_idx": i,
                "cand_label": self.target_names[self.labels[i]] if self.labels is not None else None,
                "cosine_sim": sim_scores[i]
            })

    return pd.DataFrame(results)

> Inicializamos el vectorizador

In [74]:
vectorizer = MyVectorizer(train.data, train.target, train.target_names)


<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


> Convertimos texto a indice y viceversa

In [75]:
vectorizer.get_index_from_vocab("car")

25775

In [76]:
vectorizer.get_word_from_index(25775)

'car'

> Tomamos 5 docs aleatorios y calculamos similaridad

In [77]:
vectorizer.analyze_similarity(n_queries=5, topk=5, random_state=42)

,query_idx,query_label,cand_idx,cand_label,cosine_sim
0,8754,talk.religion.misc,6552,talk.religion.misc,0.490405
1,8754,talk.religion.misc,10613,talk.religion.misc,0.481184
2,8754,talk.religion.misc,3616,talk.religion.misc,0.465348
3,8754,talk.religion.misc,8726,talk.politics.mideast,0.459895
4,8754,talk.religion.misc,3902,talk.religion.misc,0.459074
5,4965,comp.sys.mac.hardware,5830,comp.sys.mac.hardware,0.365328
6,4965,comp.sys.mac.hardware,9736,comp.sys.mac.hardware,0.361090
7,4965,comp.sys.mac.hardware,1822,comp.sys.ibm.pc.hardware,0.355371
8,4965,comp.sys.mac.hardware,2327,comp.sys.ibm.pc.hardware,0.340710
9,4965,comp.sys.mac.hardware,3408,comp.graphics,0.340530


> Vemos que para todos los docs de query (los que se usaron para buscar similares) sus labels mantienen coherencia con los labels de los candidatos por similaridad.

> En su mayoria, 4 de 5 docs pertencen a la misma categoria que el doc de query, con el restante perteneciendo a una clase conceptualmente similar.

> Solo para el documento 4899, que parece tratarse de cryptografia (sci.cript), vemos que trae 2 docs que tratan de politica / religion (que a priori no deberian estar relacionados). Este caso, de todas formas, presenta candidatos con una similaridad coseno relativamente baja.

## **2. Modelo de clasificación por prototipos**

*Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad*

In [78]:
from sklearn.metrics import classification_report, f1_score


class PrototipeModel:
  def __init__(self, vectorizer, train, test):
    self.vectorizer = vectorizer
    self.X_test = vectorizer.transform(test.data)
    self.X_train = vectorizer.get_doc_term_matrix()
    self.y_test = test.target

    self.train_set = train

  def predict(self):
    # Calculamos las simlaridades cross entre docs de test y los q ya conocemos de train
    S = cosine_similarity(self.X_test, self.X_train)  # matriz [n_test, n_train]

    # obtenemos el indice mas cercano
    nn_idx = S.argmax(axis=1)

    # asignamos prediccion = target del indice mas cercano
    y_pred_proto = self.train_set.target[nn_idx]
    y_true = self.y_test

    # report
    print(classification_report(y_true, y_pred_proto, target_names=self.train_set.target_names, zero_division=0))


In [79]:
proto_model = PrototipeModel(vectorizer, train, test)
proto_model.predict()

                          precision    recall  f1-score   support

             alt.atheism       0.37      0.51      0.43       319
           comp.graphics       0.54      0.48      0.51       389
 comp.os.ms-windows.misc       0.51      0.46      0.48       394
comp.sys.ibm.pc.hardware       0.52      0.52      0.52       392
   comp.sys.mac.hardware       0.53      0.50      0.52       385
          comp.windows.x       0.70      0.59      0.64       395
            misc.forsale       0.63      0.46      0.53       390
               rec.autos       0.41      0.58      0.48       396
         rec.motorcycles       0.63      0.52      0.57       398
      rec.sport.baseball       0.65      0.54      0.59       397
        rec.sport.hockey       0.75      0.72      0.73       399
               sci.crypt       0.55      0.59      0.57       396
         sci.electronics       0.53      0.33      0.41       393
                 sci.med       0.65      0.49      0.56       396
         

## **3. Modelos de Clasificación**

*Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.*

> Hacemos un tunning de hyperparams de ambas variantes de NB y nos quedamos con el mejor

In [80]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

# validation split para hyperparamtunning
X_tr_in, X_val, y_tr_in, y_val = train_test_split(X_train, train.target, test_size=0.2, random_state=42, stratify=train.target)

param_grid = {
    "alpha": [0.1, 0.3, 0.5, 1.0],
}

def grid_search_nb(model_class, Xtr, ytr, Xva, yva, params):
    best = {"f1": -1, "params": None, "model": None}
    for p in ParameterGrid(params):
        clf = model_class(**p)
        clf.fit(Xtr, ytr)
        pred = clf.predict(Xva)
        f1 = f1_score(yva, pred, average='macro')
        if f1 > best["f1"]:
            best = {"f1": f1, "params": p, "model": clf}
    return best

best_mnb = grid_search_nb(MultinomialNB, X_tr_in, y_tr_in, X_val, y_val, param_grid)
best_cnb = grid_search_nb(ComplementNB, X_tr_in, y_tr_in, X_val, y_val, param_grid)

print("Best f1 score MNB: ", best_mnb["f1"], best_mnb["params"])
print("Best f1 score CNB: ", best_cnb["f1"], best_cnb["params"])

Best f1 score MNB:  0.7218335502533424 {'alpha': 0.1}
Best f1 score CNB:  0.7734869204929108 {'alpha': 0.1}


> Re entrenamos con todo el train y predecimos en test

In [81]:
mnb = MultinomialNB(**best_mnb["params"]).fit(X_train, train.target)
cnb = ComplementNB(**best_cnb["params"]).fit(X_train, train.target)

pred_mnb = mnb.predict(X_test)
pred_cnb = cnb.predict(X_test)

print("=== MultinomialNB ===")
print(classification_report(test.target, pred_mnb, target_names=train.target_names, zero_division=0))
print("F1-macro (MNB):", f1_score(test.target, pred_mnb, average='macro'))

print("\n=== ComplementNB ===")
print(classification_report(test.target, pred_cnb, target_names=train.target_names, zero_division=0))
print("F1-macro (CNB):", f1_score(test.target, pred_cnb, average='macro'))

=== MultinomialNB ===
                          precision    recall  f1-score   support

             alt.atheism       0.69      0.23      0.35       319
           comp.graphics       0.68      0.69      0.69       389
 comp.os.ms-windows.misc       0.70      0.52      0.60       394
comp.sys.ibm.pc.hardware       0.61      0.75      0.67       392
   comp.sys.mac.hardware       0.77      0.69      0.73       385
          comp.windows.x       0.83      0.75      0.79       395
            misc.forsale       0.85      0.73      0.78       390
               rec.autos       0.81      0.75      0.78       396
         rec.motorcycles       0.82      0.73      0.77       398
      rec.sport.baseball       0.94      0.81      0.87       397
        rec.sport.hockey       0.60      0.93      0.73       399
               sci.crypt       0.61      0.79      0.69       396
         sci.electronics       0.72      0.57      0.64       393
                 sci.med       0.85      0.78      0.

> Ambos modelos superan ampliamente al enfoque por prototipos (el cual tenia un F1 Macro de .5), siendo el ComplementNB el mejor de los dos modelos con un F1 de .7

> Agregar modelos mejoró mucho la capacidad predictiva

## **4. Transposición de la matriz documento-termino y similaridad**

*Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".*

In [82]:
# La matriz documento‑término es X_train. La transpuesta es término‑documento.
# Construimos vectores de palabras y buscamos top‑k similares con coseno.
from sklearn.metrics.pairwise import cosine_similarity

doc_term_matrix = vectorizer.get_doc_term_matrix()
print("Shape doc-term:", doc_term_matrix.shape)

W = doc_term_matrix.T  # [n_terms, n_docs]
print("Shape término‑documento:", W.shape)

Shape doc-term: (11314, 101631)
Shape término‑documento: (101631, 11314)


In [83]:
# Elegimos manualmente 5 términos relevantes/interpretables del vocabulario
chosen_words = [
    "windows", "space", "hockey", "encryption", "religion"
]
existing = [w for w in chosen_words if w in vectorizer.vocabulary_]
missing = [w for w in chosen_words if w not in vectorizer.vocabulary_]
print("Elegidas (en vocab):", existing, "| Fuera de vocab:", missing)

Elegidas (en vocab): ['windows', 'space', 'hockey', 'encryption', 'religion'] | Fuera de vocab: []


In [84]:

word_to_idx = {w: vectorizer.get_index_from_vocab(w) for w in existing}

def topk_sim_words(W, idx, k=5):
    # Similaridad coseno entre una palabra y todas las demás
    sims = cosine_similarity(W[idx], W).ravel()
    sims[idx] = -np.inf
    top_idx = np.argpartition(-sims, kth=k)[:k]
    top_idx = top_idx[np.argsort(-sims[top_idx])]
    return top_idx, sims[top_idx]

In [88]:
TOPK_WORDS = 5
rows = []
for w in existing:
    idx = word_to_idx[w]
    top_idx, top_sims = topk_sim_words(W, idx, k=TOPK_WORDS)
    for rank, (ti, s) in enumerate(zip(top_idx, top_sims), start=1):
        rows.append({
            "query_word": w,
            "rank": rank,
            "cand_word": vectorizer.get_word_from_index(ti),
            "cosine_sim": float(s)
        })

df_sim_words = pd.DataFrame(rows)
df_sim_words

,query_word,rank,cand_word,cosine_sim
0,windows,1,dos,0.303705
1,windows,2,ms,0.232047
2,windows,3,microsoft,0.221937
3,windows,4,nt,0.214015
4,windows,5,for,0.192976
5,space,1,nasa,0.330422
6,space,2,seds,0.296643
7,space,3,shuttle,0.292845
8,space,4,enfant,0.280270
9,space,5,seti,0.246494


> Se observa similaridad tambien entre las palabras candidatas y las encontradas por similaridad coseno! En el caso de windows, se ve que se trataba la palabra en el contexto de la informatica.

> En el ejemplo de "space" se encuentran palabras relacionadas con misiones espaciales o bien observatorios espaciales. para el resto de palabras, sus candidatos tambien son relavantes en terminos de similaridad.